# 새싹 해커톤
## 나란히 - 장애인 인식 개선을 위한 생성형AI

### 1. PDF를 읽어보기

#### 1-1) PyPDF2, PyMuPDF

In [26]:
!pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [27]:
import PyPDF2

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
    return text

import fitz  # PyMuPDF


def read_pdf_with_pymupdf(file_path):
    doc = fitz.open(file_path)
    text = ''
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # 페이지 불러오기
        text += page.get_text()  # 페이지에서 텍스트 추출
    return text


pdf_text = read_pdf_with_pymupdf('./dataset/[PPT] 직장 내 장애인 인식개선 교육 표준교안 PPT(국문)_수정.pdf')
print(pdf_text)

고용장려금
경증남성
경증여성
중증남성
중증여성



In [29]:
if __name__ == "__main__":
    file_path = './dataset/[PPT] 직장 내 장애인 인식개선 교육 표준교안 PPT(국문)_수정.pdf'

    pdf_text = read_pdf(file_path)
    print(pdf_text)

고용장려금경증남성 경증여성
중증남성 중증여성



#### OCR인식 없이는 잘 읽히지 않는 것 같다.

python에서 지원하는 PDF reader를 사용해보았지만...  
드래그로 긁을 수 있는 형식이 아니라 그런지 잘 읽히진 않는다.

### 1-2) OCR reader (pytesseract)

In [30]:
"""
Requirements: `pip install pytesseract pdf2image pillow`
"""

# OCR을 사용

import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os

# Tesseract 경로 설정 (윈도우의 경우)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# PDF 파일 경로 설정
pdf_path = './dataset/[PPT] 직장 내 장애인 인식개선 교육 표준교안 PPT(국문)_수정.pdf'

# PDF를 이미지로 변환, 해상도 300dpi(고화질)로 조정
pages = convert_from_path(pdf_path, 300) 

# 각 페이지에서 텍스트 추출
def ocr_from_pdf(pages):
    text = ''
    for num, page in enumerate(pages):
        # 이미지를 일시적으로 저장
        temp_image_path = f"temp_page_{num+1}.png"
        page.save(temp_image_path, 'PNG')
        
        # 이미지에서 텍스트 추출
        page_text = pytesseract.image_to_string(Image.open(temp_image_path), lang='kor+eng')
        text += page_text
        
        # 일시적 이미지 파일 삭제
        os.remove(temp_image_path)
        
    return text

In [32]:
# 텍스트 추출
pdf_text = ocr_from_pdf(pages)
print(pdf_text)

KeyboardInterrupt: 

In [34]:
from pprint import pprint as pp
pp(pdf_text)

('가 보입니다\n'
 '\n'
 '장애인 고용, 기업에게 어떤 의미가 있을까\n'
 '\n'
 '종료 이해하기\n'
 '\n'
 '느\n'
 'Lo\n'
 '\n'
 '16\n'
 'oll\n'
 '\n'
 '~\n'
 '100\n'
 '\n'
 'Al 일하기, 이렇게 하면 잘 할 수 있습니다\n'
 '\n'
 '한\n'
 '=\n'
 '\n'
 '장애인 고용,\n'
 '\n'
 '기업에게 어떤 의미가 있을까\n'
 '\n'
 '0\n'
 '\n'
 '애인고용\n'
 '\n'
 'iE\n'
 '\n'
 '기업 사례\n'
 '\n'
 '으로 담담히 No.1 게임 기업의 SHO! 되다!\n'
 '\n'
 '@.NEXON                                                             넥슨컴즈의 '
 '장애인\n'
 '\n'
 'ASS 우대도 없었던 게임 산업\n'
 '\n'
 '위기 속 장애인 고응의 들파구       ~ 애에마테테\n'
 '__                                                                  이중 중증장애인 '
 '비율 73% 09\n'
 '\n'
 '201년8넥슨커뮤니케이션즈(넥슨컴지를 설립하고\n'
 '\n'
 '넥슨의 게임 SO} 서비스지원과 고객 지원 업무를 담당함\n'
 '\n'
 '4개월간의 SSeS 거친 장애인 23명이\n'
 '처음으로산업에입사하게됨\n'
 '\n'
 '“AS 홈페이지 관리, 게임 상담, 모니터링 등\n'
 '\n'
 '상시 근로자 2.691명 / 장애인\n'
 '\n'
 '근로자 39S (2.04%)\n'
 '\n'
 '“2018.12 기준\n'
 '전문가와 함께\n'
 '\n'
 '고용 대책 마련\n'
 '\n'
 '장애물 없는 생활환경 (Barrier Free)\n'
 '자체 인증 제도를 각 사업장에 도입\n'
 '\n'
 '상담심리사를 상주시켜\n'
 '갈등, 대인관계 등 고중을 해결\n'
 '\n'
 '장매인 고

In [16]:
# 텍스트를 txt 파일로 저장
def save_text_to_file(text, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

# 저장할 파일 경로 설정
txt_file_path = 'extracted_text.txt'
save_text_to_file(pdf_text, txt_file_path)
print(f"Text saved to {txt_file_path}")

Text saved to extracted_text.txt


읽히긴 하지만 오타도 많고 의미를 알아보기가 쉽지 않았다.  
이미지 전처리를 통해 

In [23]:
import cv2
import numpy as np
from PIL import Image

# 이미지 전처리
def preprocess_image(image):
    # 이미지를 그레이스케일로 변환
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    # 이진화
    _, binary_image = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # 노이즈 제거
    processed_image = cv2.medianBlur(binary_image, 3)
    return Image.fromarray(processed_image)


# 각 페이지에서 텍스트 추출
def ocr_from_pdf(pages):
    text = ''
    for num, page in enumerate(pages):
        # 이미지를 일시적으로 저장
        temp_image_path = f"temp_page_{num+1}.png"
        processed_page = preprocess_image(page)
        processed_page.save(temp_image_path, 'PNG')
        
        # 이미지에서 텍스트 추출
        page_text = pytesseract.image_to_string(Image.open(temp_image_path), lang='kor+eng')
        text += page_text
        
        # 일시적 이미지 파일 삭제
        os.remove(temp_image_path)
        
    return text


In [24]:
pdf_text_preprocessed = ocr_from_pdf(pages)
print(pdf_text_preprocessed)

txt_file_path = 'preprocessed_OCR.txt'
save_text_to_file(pdf_text_preprocessed, txt_file_path)
print(f"Text saved to {txt_file_path}")

장매유형별 모듈

한국장애인고용공단 FP

Contents

0. 장애이해하7|(공통)
1. 일터 엿보기

2. 미해하기

3. 함께 일하7|

4. 71억해주세요/

5. 마무리

0. St0H 이해하기                                                                          한국장야인고8공단 FL

장매인복지법

장애인은 신체적ㆍ정신적 장애로 인하여

(0
오랫동안 일상 및 사회생활에
상당한 제약을 받는 자

장애인이란 신체 또는 정
장기간에 걸쳐 직업생활에
상당한 제약을 받는 자

0. 장애 이해하기
한국장애인고용공단

ㆍ지체장애
>뇌병변장애                                             끼적

» 시각장애                                                강매

0. Zt0H 이해하기                                                                          eagqensan AP

장애등급제 폐지

복/끼서비스 NAS 위한 장애인

MOSES 장매정도로 변경

ㆍ의학적 기준(1~6급)                       ㆍ심한 장애인(1~3급)
ㆍ심하지 않은 장애인(4~6급)

0. St0H 이해하기                                                                         한국장애인고8공단 FP

AAA @

록현황 / 20194 128 기준)

0. St0H 이해하기                                                                          한국장야인고8공단 FL

출생시원인
14%

원인불명 선전적원인 |
54% —N%

후천적 질환

56.0%

출처 : 2017년 장애인 실태조사

누구도 장애인이 될 가능성에서 ASSN 않습니다

0. 장애 이해하기    

In [29]:
"""
Requirements: `pip install pyspellchecker`
"""

from spellchecker import SpellChecker

spell = SpellChecker(language='ko')

def correct_spelling(text):
    corrected_text = []
    words = text.split()
    for word in words:
        corrected_word = spell.correction(word)
        corrected_text.append(corrected_word)
    return ' '.join(corrected_text)

# 텍스트 추출 후 철자 교정
# pdf_text = ocr_from_pdf(pages)
corrected_text = correct_spelling(pdf_text_preprocessed)
print(corrected_text)

ValueError: The provided dictionary language (ko) does not exist!

In [37]:
"""
Requirements: `pip install korpora`
"""

from Korpora import Korpora

# 사용할 데이터셋 다운로드
Korpora.fetch('kowikitext')

# 데이터 로드
kowikitext = Korpora.load('kowikitext')

# 단어 목록 추출
words = set()

# KoWikiText 데이터에서 단어 추출
for text in kowikitext.train:
    words.update(text.split())

# 단어 목록을 파일로 저장
with open('korean_words.txt', 'w', encoding='utf-8') as f:
    for word in words:
        f.write(f"{word}\n")


[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repos

AttributeError: 'SentencePair' object has no attribute 'split'

In [39]:
from Korpora import Korpora

# 사용할 데이터셋 다운로드
Korpora.fetch('kowikitext')

# 데이터 로드
kowikitext = Korpora.load('kowikitext')

[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repos

In [40]:
import re

# 한글 단어만 추출하기 위한 정규식 패턴
hangul_pattern = re.compile(r'[가-힣]+')

# 단어 목록 추출
words = set()

# KoWikiText 데이터에서 단어 추출
for sentence in kowikitext.train:
    for word in sentence.text.split():
        if hangul_pattern.fullmatch(word):
            words.add(word)

In [41]:
# 단어 목록을 파일로 저장
with open('korean_words.txt', 'w', encoding='utf-8') as f:
    for word in words:
        f.write(f"{word}\n")

# 결과 확인
print(f"총 {len(words)}개의 한국어 단어가 추출되었습니다.")

총 4426308개의 한국어 단어가 추출되었습니다.


In [43]:
from spellchecker import SpellChecker

# 한국어 단어 목록 파일 경로
dictionary_path = 'korean_words.txt'
index_path = 'korean_dictionary.gz'

# 한국어 단어 목록을 불러와서 인덱스 파일 생성
spell = SpellChecker(language=None)
with open(dictionary_path, 'r', encoding='utf-8') as file:
    words = file.read().splitlines()
    spell.word_frequency.load_words(words)

# 인덱스 파일로 저장
spell.export(index_path, gzipped=True)


In [45]:
from spellchecker import SpellChecker

# 생성된 인덱스 파일 설정
spell = SpellChecker(language=None, local_dictionary='korean_dictionary.gz')

# 철자 교정 예제
text = "장애인 인식 개선을 위한 방법은 무엇인가요?"
words = text.split()
corrected_words = [spell.correction(word) for word in words]
print(corrected_words)
corrected_text = ' '.join(corrected_words)
print(corrected_text)


['장애인', '인식', '개선을', '위한', '방법은', '무엇인가요']
장애인 인식 개선을 위한 방법은 무엇인가요


### 1-3) pptx Reader

In [2]:
!pip install python-pptx

     -------------------------------------- 471.6/471.6 kB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 11.6 MB/s eta 0:00:00
     -------------------------------------- 159.9/159.9 kB 9.4 MB/s eta 0:00:00


In [3]:
"""
Requirements: pip install python-pptx
"""

from pptx import Presentation
import pandas as pd

file_name = "[PPT] 직장 내 장애인 인식개선 교육 표준교안 PPT(국문)_수정"
prs = Presentation(f"dataset/{file_name}.pptx")

# def: extract text from slide
def extract_text_from_slide(slide):
    text = ""
    for shape in slide.shapes:
        if hasattr(shape, "text_frame") and shape.text_frame:
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    text += run.text + " "
    return text.strip()

# Extract Text from all slides
all_slides_text = []
for slide_num, slide in enumerate(prs.slides, start=1):
    slide_content = {
        "Slide Number": slide_num,
        "Text": extract_text_from_slide(slide)
    }
    all_slides_text.append(slide_content)

# convert the list to a DataFrame
all_slides_text_df = pd.DataFrame(all_slides_text)

csv_file_path = f"extract_text_{file_name}.csv"
all_slides_text_df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

print(f"CSV file saved to {csv_file_path}")

CSV file saved to extract_text_[PPT] 직장 내 장애인 인식개선 교육 표준교안 PPT(국문)_수정.csv


In [4]:
example_text = all_slides_text_df['Text'].iloc[4]
example_text

'장애인 업무의 한계를 없애다   상시  근로자  100,016 명  /  장애인  근로자  1,386 명 (1.54%)  * 2018.12 월 기준 장애물 없는 생활환경  (Barrier Free) 자체 인증 제도를 각 사업장에 도입  상담심리사를 상주시켜  직무갈등 ,  대인관계 등 고충을 해결  업무에 집중할 환경을 마련  장애인 고객의 입장에서  바라본   기술개발에도 적극 참여  장애인은   단순 업무만 해야 하나 ? ‘ 삼성전자 맞춤형 인재 ’ 로 성장하는데 있어서  비장애인과 장애인의 구분은 의미가 없다고 판단 , 직무교육을 거쳐 고용된 장애인은 각 사업장에서  전기 · 전자 ,  정보기술 ,  기계디자인 분야에서 근무 중 장애인고용 기업 사례 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련'

In [14]:
from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer


# 모델과 토크나이저 로드
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)

# 입력 텍스트를 모델 입력 형식으로 변환
input_text = f"split sentences: {example_text}"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 모델로 문장 생성
outputs = model.generate(input_ids, max_new_tokens=200, num_beams=5, early_stopping=True)

# 결과 디코딩 및 출력
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
print(result)

[{'generated_text': '                                                        '}]


In [10]:
import spacy

# spaCy 모델 로드
nlp = spacy.blank('ko')
nlp.add_pipe('sentencizer')

# 주어진 텍스트
text = '''장애인 업무의 한계를 없애다 상시 근로자 100,016 명 / 장애인 근로자 1,386 명 (1.54%) * 2018.12 월 기준 장애물 없는 생활환경 
(Barrier Free) 자체 인증 제도를 각 사업장에 도입 상담심리사를 상주시켜 직무갈등 , 대인관계 등 고충을 해결 
업무에 집중할 환경을 마련 장애인 고객의 입장에서 바라본 기술개발에도 적극 참여 장애인은 단순 업무만 해야 하나 ? 
‘ 삼성전자 맞춤형 인재 ’ 로 성장하는데 있어서 비장애인과 장애인의 구분은 의미가 없다고 판단 , 직무교육을 거쳐 고용된 장애인은 
각 사업장에서 전기 · 전자 , 정보기술 , 기계디자인 분야에서 근무 중 장애인고용 기업 사례 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련'''

# 텍스트를 문장 단위로 분리
doc = nlp(text)
sentences = [sent.text.strip() + '.' for sent in doc.sents]

# 결과 출력
for sentence in sentences:
    print(sentence)

ImportError: The Korean tokenizer ("spacy.ko.KoreanTokenizer") requires [mecab-ko](https://bitbucket.org/eunjeon/mecab-ko/src/master/README.md), [mecab-ko-dic](https://bitbucket.org/eunjeon/mecab-ko-dic), and [natto-py](https://github.com/buruzaemon/natto-py)

In [15]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# 문장 단위로 분리
sentences = sent_tokenize(example_text)

# 결과 출력
for sentence in sentences:
    print(sentence + '.')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dm705\AppData\Roaming\nltk_data...


장애인 업무의 한계를 없애다   상시  근로자  100,016 명  /  장애인  근로자  1,386 명 (1.54%)  * 2018.12 월 기준 장애물 없는 생활환경  (Barrier Free) 자체 인증 제도를 각 사업장에 도입  상담심리사를 상주시켜  직무갈등 ,  대인관계 등 고충을 해결  업무에 집중할 환경을 마련  장애인 고객의 입장에서  바라본   기술개발에도 적극 참여  장애인은   단순 업무만 해야 하나 ?.
‘ 삼성전자 맞춤형 인재 ’ 로 성장하는데 있어서  비장애인과 장애인의 구분은 의미가 없다고 판단 , 직무교육을 거쳐 고용된 장애인은 각 사업장에서  전기 · 전자 ,  정보기술 ,  기계디자인 분야에서 근무 중 장애인고용 기업 사례 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련.


[nltk_data]   Unzipping tokenizers\punkt.zip.


In [16]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 모델과 토크나이저 로드
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 주어진 텍스트
text = (
    "장애인 업무의 한계를 없애다 상시 근로자 100,016 명 / 장애인 근로자 1,386 명 (1.54%) * 2018.12 월 기준 장애물 없는 생활환경 "
    "(Barrier Free) 자체 인증 제도를 각 사업장에 도입 상담심리사를 상주시켜 직무갈등 , 대인관계 등 고충을 해결 "
    "업무에 집중할 환경을 마련 장애인 고객의 입장에서 바라본 기술개발에도 적극 참여 장애인은 단순 업무만 해야 하나 ? "
    "‘ 삼성전자 맞춤형 인재 ’ 로 성장하는데 있어서 비장애인과 장애인의 구분은 의미가 없다고 판단 , 직무교육을 거쳐 고용된 장애인은 "
    "각 사업장에서 전기 · 전자 , 정보기술 , 기계디자인 분야에서 근무 중 장애인고용 기업 사례 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련"
)

# 입력 텍스트에 프롬프트 추가
input_text = f"split sentences: {text}"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# 모델로 문장 생성
outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)

# 결과 디코딩 및 출력
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
sentences = decoded_output.split('. ')
for sentence in sentences:
    print(sentence.strip() + '.')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


.


In [17]:
!pip install kss

     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 1.1/1.1 MB 35.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
     ---- ----------------------------------- 2.9/26.4 MB 61.7 MB/s eta 0:00:01
     -------- ------------------------------- 5.3/26.4 MB 56.3 MB/s eta 0:00:01
     ------------ --------------------------- 8.5/26.4 MB 60.9 MB/s eta 0:00:01
     --------------- ----------------------- 10.7/26.4 MB 59.5 MB/s eta 0:00:01
     --------------- ----------------------- 10.8/26.4 MB 43.7 MB/s eta 0:00:01
     ---------------- ---------------------- 11.

  You can safely remove it manually.


In [20]:
from kss import Kss

split_sentences = Kss("split_sentences")
result = split_sentences(example_text)
len(result)


2

In [21]:
!pip install kiwipiepy

In [24]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.split_into_sents(example_text)


[Sentence(text='장애인 업무의 한계를 없애다   상시  근로자  100,016 명  /  장애인  근로자  1,386 명 (1.54%)  * 2018.12 월 기준 장애물 없는 생활환경  (Barrier Free) 자체 인증 제도를 각 사업장에 도입  상담심리사를 상주시켜  직무갈등 ,  대인관계 등 고충을 해결  업무에 집중할 환경을 마련  장애인 고객의 입장에서  바라본   기술개발에도 적극 참여  장애인은   단순 업무만 해야 하나 ? ‘ 삼성전자 맞춤형 인재 ’', start=0, end=255, tokens=None, subs=[]),
 Sentence(text='로 성장하는데 있어서  비장애인과 장애인의 구분은 의미가 없다고 판단 , 직무교육을 거쳐 고용된 장애인은 각 사업장에서  전기 · 전자 ,  정보기술 ,  기계디자인 분야에서 근무 중 장애인고용 기업 사례 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련', start=256, end=402, tokens=None, subs=[])]